In [6]:
from pathlib import Path

base_dir = Path("/data1/ujiie/wiki")
file_set = base_dir.glob("**/wiki_*")

In [3]:
# create alias table

from xml.sax.saxutils import unescape
from urllib.parse import unquote_to_bytes
import re
import csv
import json

title2id = {}
id_set = set()
alias_set = set()
redirect_cnt = 0
title_cnt = 0
with open('/data1/ujiie/WikiDump/jawiki-20190120-title2pageid.json', 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
            
        line = json.loads(line)
        if line['is_redirect']:
            if line['redirect_to']['page_id'] is None:
                continue
                
            if int(line['redirect_to']['page_id']) > 3894437:
                print(line)
                continue
            alias_set.add((line['redirect_to']['page_id'], line['title']))
            title2id[line['title']] = line['redirect_to']['page_id']
            redirect_cnt += 1
        else:
            if line['page_id'] is None:
                continue
            if int(line['page_id']) > 3894437:
                print(line)
                continue
            id_set.add(line['page_id'])
            title2id[line['title']] = line['page_id']
            alias_set.add((line['page_id'], line['title']))
            title_cnt += 1
""" 
link_cnt = 0
with open("/data1/ujiie/wiki_resource/alias_table.tsv", "w") as fout:
    writer = csv.writer(fout)
    for fn in file_set:
        with open(fn, 'r') as f:
            for line in f:
                line = line.rstrip()
                if not line:
                    continue
                    
                line = json.loads(line)["text"].split("\n")
                line = [unescape(l) for l in line]
                line = [l for l in line if "<a" in l]
                line = ''.join(line)
                matches = re.findall('<a href="(.*?)">(.*?)</a>', line)
                for m in matches:
                    href = unquote_to_bytes(m[0]).decode()
                    mention = m[1]
                    if href == "" or href.startswith("http") or mention == "":
                        continue
                    
                    
                    if href not in title2id:
                        cnt += 1
                    else:
                        output = (title2id[href], mention)
                        if output in alias_set:
                            continue
                        alias_set.add((title2id[href], mention))
                        link_cnt += 1

    for page_id, title in list(alias_set):
        writer.writerow([page_id, title])
print(redirect_cnt, title_cnt, link_cnt)
"""

{'page_id': 3894454, 'title': 'KO_SHIBASAKI_ALL_TIME_BEST', 'is_redirect': False}
{'page_id': 3894452, 'title': 'WORLDISTA', 'is_redirect': False}
{'page_id': 3894438, 'title': 'シルヴィア・ルウェイン・デイヴィス', 'is_redirect': False}
{'page_id': 3894463, 'title': 'ハノイ都市鉄道2A号線', 'is_redirect': False}
{'page_id': 3894444, 'title': 'モナ・グルート', 'is_redirect': False}
{'page_id': 3894449, 'title': 'ルファック', 'is_redirect': False}
{'page_id': 3894464, 'title': '兵部行遠', 'is_redirect': False}
{'page_id': 3894445, 'title': '宮澤那名子', 'is_redirect': False}
{'page_id': 3894458, 'title': '山脇秀夫', 'is_redirect': False}
{'page_id': 3894451, 'title': '時里二郎', 'is_redirect': False}
{'page_id': 3894461, 'title': '本宮映画劇場', 'is_redirect': False}
{'page_id': 3894450, 'title': '村上信五のスポーツ奇跡の瞬間アワード', 'is_redirect': False}
{'page_id': 3894457, 'title': '津島隆太', 'is_redirect': False}
{'page_id': 3894465, 'title': '羽鳥×宮本_福岡好いとぉ', 'is_redirect': False}


' \nlink_cnt = 0\nwith open("/data1/ujiie/wiki_resource/alias_table.tsv", "w") as fout:\n    writer = csv.writer(fout)\n    for fn in file_set:\n        with open(fn, \'r\') as f:\n            for line in f:\n                line = line.rstrip()\n                if not line:\n                    continue\n                    \n                line = json.loads(line)["text"].split("\n")\n                line = [unescape(l) for l in line]\n                line = [l for l in line if "<a" in l]\n                line = \'\'.join(line)\n                matches = re.findall(\'<a href="(.*?)">(.*?)</a>\', line)\n                for m in matches:\n                    href = unquote_to_bytes(m[0]).decode()\n                    mention = m[1]\n                    if href == "" or href.startswith("http") or mention == "":\n                        continue\n                    \n                    \n                    if href not in title2id:\n                        cnt += 1\n                   

In [5]:
# create id2title table

import csv
import re
import json
from xml.sax.saxutils import unescape
from urllib.parse import unquote_to_bytes

title2id = {}
with open("/data1/ujiie/wiki_resource/id_title_description.jsonl", "w") as fout:
    for fn in file_set:
        with open(fn, 'r') as f:
            for line in f:
                line = line.rstrip()
                if not line:
                    continue
                    
                
                line = json.loads(line)
                
                # remove redirect page
                if int(line['id']) not in id_set:
                    continue
                
                _id = line['id']
                title = line['title']
                
                text = line['text']
                text = text.split("\n")[0]
                text = unescape(text)
                text = re.sub('<a href="(.*?)">(.*?)</a>', r'\2', text)
                
                title2id[line['title']] = line['id']
                
                fout.write(json.dumps({"id": _id, "title": title, "description": text}) + '\n')

In [30]:
len(alias_set)

2366824

In [7]:
# create training data (all)

from xml.sax.saxutils import unescape
from urllib.parse import unquote_to_bytes
import re
import json

cnt = 0
alias_set = set()
with open("/data1/ujiie/wiki_resource/training_data.jsonl", "w") as fout:
    for fn in file_set:
        sent = []
        with open(fn, 'r') as f:
            for line in f:
                line = line.rstrip()
                if not line:
                    continue
                
                line = json.loads(line)
                doc = line["text"].split("\n")
                doc = [''] + [unescape(s) for s in doc if s != ''] + [""]
                plain_doc = [re.sub('<a href="(.*?)">(.*?)</a>', r'\2', t) for t in doc]
                for idx in range(1, len(doc)-1):
                    sent = doc[idx]
                    ite = re.finditer('<a href="(.*?)">(.*?)</a>', sent)
                    for it in ite:
                        link = unquote_to_bytes(it.groups()[0]).decode()
                        if link not in title2id:
                            continue
                        link = title2id[link]
                        mention = it.groups()[1]
                        start, end = it.span()
                        left_context = sent[:start]
                        left_context = re.sub('<a href="(.*?)">(.*?)</a>', r'\2', left_context)
                        right_context = sent[end:]
                        right_context = re.sub('<a href="(.*?)">(.*?)</a>', r'\2', right_context)
                        output = {
                            "pre_sent": plain_doc[idx-1],
                            "left_context": left_context,
                            "right_context": right_context + plain_doc[idx+1],
                            "post_sent": plain_doc[idx+1],
                            "mention": mention,
                            "linkpage_id": link
                        }
                        fout.write(json.dumps(output, ensure_ascii=False) + '\n')

In [30]:
# create entity prior

from xml.sax.saxutils import unescape
from urllib.parse import unquote_to_bytes
import re
import csv
from collections import defaultdict
import pickle
cnt = 0

entity_prior = defaultdict(int)
total = 0

for fn in file_set:
    with open(fn, 'r') as f:
        for line in f:
            line = line.rstrip()
            if not line or line.startswith("<doc") or line.startswith("</doc"):
                continue
            line = unescape(line)
            matches = re.findall('<a href="(.*?)">(.*?)</a>', line)
            for m in matches:
                href = unquote_to_bytes(m[0]).decode()
                mention = m[1]
                if href == "" or href.startswith("http") or mention == "":
                    continue
                if href not in title2id:
                    continue
                    
                entity_prior[title2id[href]] += 1
                total += 1

"""
keys = list(entity_prior.keys())
for key in keys:
    entity_prior[key] = entity_prior[key] / total
"""
with open("/data1/ujiie/wiki_resource/entity_prior.pkl", "wb") as f:
    pickle.dump(entity_prior, f)
                

In [32]:
# create mention-entity prior

from xml.sax.saxutils import unescape
from urllib.parse import unquote_to_bytes
import re
import csv
from collections import defaultdict
import pickle
cnt = 0

mention_prior = {}

for fn in file_set:
    with open(fn, 'r') as f:
        for line in f:
            line = line.rstrip()
            if not line:
                continue
                
            line = json.loads(line)["text"].split("\n")
            line = [unescape(l) for l in line]
            line = [l for l in line if "<a" in l]
            line = ''.join(line)
            
            matches = re.findall('<a href="(.*?)">(.*?)</a>', line)
            for m in matches:
                href = unquote_to_bytes(m[0]).decode()
                mention = m[1]
                if href == "" or href.startswith("http") or mention == "":
                    continue
                    
                if href not in title2id:
                    continue
                    
                if mention not in mention_prior:
                    mention_prior[mention] = defaultdict(int)
                    
                mention_prior[mention][title2id[href]] += 1
                
for title, _id in title2id.items():
        if title not in mention_prior:
            mention_prior[title] = defaultdict(int)
        mention_prior[title][title2id[title]] += 1

In [34]:
keys = list(mention_prior.keys())
for key in keys:
    a = sorted([(a[1], a[0]) for a in mention_prior[key].items()], reverse=True)
    mention_prior[key] = a
with open("/data1/ujiie/wiki_resource/mention_prior.pkl", "wb") as f:
    pickle.dump(mention_prior, f)

In [11]:
len(title2id)

1830861

In [9]:
mention_prior["工藤優作"]

[(1, '1375874')]

In [15]:
"未来日記-ANOTHER:WORLD-" in title2id

False

In [36]:
path = "/data1/ujiie/WikiDump/jawiki-20190120-pages-articles-multistream-index.txt"
with open(path, 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
            
        print(line)
        break

695:1:Wikipedia:アップロードログ 2004年4月


In [12]:
import numpy as np
index = np.array([0])
with open('/data1/ujiie/wiki_resource/sample_preprocessed.jsonl', 'r') as f:
    for line in f:
        index = np.append(index, index[-1] + len(line))
index = index[:-1]
print(index)
np.save('/data1/ujiie/wiki_resource/sample_preprocessed_index.npy', index)

[        0      1858      3715 ... 609247705 609254688 609265353]


In [16]:
with open('/data1/ujiie/wiki_resource/sample_preprocessed.jsonl', 'r') as f:
    f.seek(index[1])
    lines = f.read(index[2] - index[1])[:-1]
    
    f.seek(index[2])
    lines = f.read(index[3] - index[2])[:-1]
lines

'{"pre_sent": "", "left_context": "\\u30a2\\u30f3\\u30d1\\u30b5\\u30f3\\u30c9 (&\\u3001\\u82f1\\u8a9e\\u540d\\uff1a) \\u3068\\u306f\\u4e26\\u7acb\\u52a9\\u8a5e\\u300c\\u2026\\u3068\\u2026\\u300d\\u3092\\u610f\\u5473\\u3059\\u308b\\u8a18\\u53f7\\u3067\\u3042\\u308b\\u3002\\u30e9\\u30c6\\u30f3\\u8a9e\\u306e \\u306e", "right_context": "\\u3067\\u3001Trebuchet MS\\u30d5\\u30a9\\u30f3\\u30c8\\u3067\\u306f\\u3001\\u3068\\u8868\\u793a\\u3055\\u308c \\"et\\" \\u306e\\u5408\\u5b57\\u3067\\u3042\\u308b\\u3053\\u3068\\u304c\\u5bb9\\u6613\\u306b\\u308f\\u304b\\u308b\\u3002ampersa\\u3001\\u3059\\u306a\\u308f\\u3061 \\"and per se and\\"\\u3001\\u305d\\u306e\\u610f\\u5473\\u306f\\"and [the symbol which] by itself [is] and\\"\\u3067\\u3042\\u308b\\u3002\\u6b74\\u53f2.", "post_sent": "\\u6b74\\u53f2.", "mention": "\\u5408\\u5b57", "linkpage_id": 90949, "left_ctxt_tokens": ["\\u30a2\\u30f3", "##\\u30d1", "##\\u30b5\\u30f3\\u30c9", "(", "##&", "##\\u3001", "\\u82f1\\u8a9e", "\\u540d", ":)", "\\u3068", "\